# CASE: BOMBOM - Advanced Analytics IBM + Vale

## Realizado por: Sabrina Otoni da Silva

## Etapa de feature engineering

In [1]:
from pathlib import Path

import pandas as pd

from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

import joblib

In [2]:
datapath = Path('../data')

In [3]:
data_model = pd.read_excel(f'{datapath}/01_raw/registros-prod.xlsx')
data_predictive = pd.read_excel(f'{datapath}/01_raw/analise-preditiva.xlsx')
data_prescriptive = pd.read_excel(f'{datapath}/01_raw/analise-prescritiva.xlsx')

In [4]:
categorical_hot_transformer = Pipeline(steps=[
    ('onehot', OneHotEncoder(handle_unknown='ignore', sparse_output=False, drop='first'))
])

categorical_ordinal_transformer = Pipeline(steps=[
    ('ordinal', OrdinalEncoder(handle_unknown='error'))
])

preprocessor_hot = ColumnTransformer(
    transformers=[
        ('cat', categorical_hot_transformer, ['VAR_2'])
    ])

preprocessor_ordinal = ColumnTransformer(
    transformers=[
        ('cat', categorical_ordinal_transformer, ['VAR_2'])
    ])

preprocessor_hot.fit(data_model)
preprocessor_ordinal.fit(data_model)

ColumnTransformer(transformers=[('cat',
                                 Pipeline(steps=[('ordinal',
                                                  OrdinalEncoder())]),
                                 ['VAR_2'])])

In [5]:
preprocessorpath = Path('../preprocessing')

In [6]:
joblib.dump(preprocessor_hot, preprocessorpath / 'preprocessor_hot.pkl')
joblib.dump(preprocessor_ordinal, preprocessorpath / 'preprocessor_ordinal.pkl')

['..\\preprocessing\\preprocessor_ordinal.pkl']

In [7]:
data_model_hot_encoded = preprocessor_hot.transform(data_model)
data_predictive_hot_encoded = preprocessor_hot.transform(data_predictive)
data_prescriptive_hot_encoded = preprocessor_hot.transform(data_prescriptive)

In [8]:
data_model_ordinal_encoded = preprocessor_ordinal.transform(data_model)
data_predictive_ordinal_encoded = preprocessor_ordinal.transform(data_predictive)
data_prescriptive_ordinal_encoded = preprocessor_ordinal.transform(data_prescriptive)

In [9]:
data_model_hot_encoded = pd.DataFrame(data_model_hot_encoded, columns=preprocessor_hot.get_feature_names_out())
data_model_hot_encoded.columns = [col.split("__")[-1] for col in data_model_hot_encoded.columns]
data_model_hot_encoded = pd.concat([data_model, data_model_hot_encoded], axis=1)
data_model_hot_encoded.drop('VAR_2', axis=1, inplace=True)
data_model_hot_encoded.head()

,QTD_CHOC,VAR_1,PESO_BOMBOM,VAR_2_B,VAR_2_C
0,320.15,0.88,10.144919,0.0,0.0
1,320.14,1.55,11.547027,0.0,0.0
2,284.85,2.97,11.112109,0.0,0.0
3,263.92,2.36,9.900123,0.0,1.0
4,276.22,2.98,10.079822,0.0,1.0


In [10]:
data_predictive_hot_encoded = pd.DataFrame(data_predictive_hot_encoded, columns=preprocessor_hot.get_feature_names_out())
data_predictive_hot_encoded.columns = [col.split("__")[-1] for col in data_predictive_hot_encoded.columns]
data_predictive_hot_encoded = pd.concat([data_predictive, data_predictive_hot_encoded], axis=1)
data_predictive_hot_encoded.drop('VAR_2', axis=1, inplace=True)
data_predictive_hot_encoded.head()

,QTD_CHOC,VAR_1,PESO_BOMBOM,VAR_2_B,VAR_2_C
0,286.63,2.72,NaN,0.0,0.0
1,257.27,1.97,NaN,1.0,0.0
2,353.16,2.64,NaN,0.0,0.0
3,317.78,2.76,NaN,0.0,0.0
4,248.10,2.63,NaN,0.0,1.0


In [11]:
data_prescriptive_hot_encoded = pd.DataFrame(data_prescriptive_hot_encoded, columns=preprocessor_hot.get_feature_names_out())
data_prescriptive_hot_encoded.columns = [col.split("__")[-1] for col in data_prescriptive_hot_encoded.columns]
data_prescriptive_hot_encoded = pd.concat([data_prescriptive, data_prescriptive_hot_encoded], axis=1)
data_prescriptive_hot_encoded.drop('VAR_2', axis=1, inplace=True)
data_prescriptive_hot_encoded.head()

,QTD_CHOC,VAR_1,VAR_2_B,VAR_2_C
0,NaN,2.07,0.0,1.0
1,NaN,2.94,1.0,0.0
2,NaN,2.53,1.0,0.0
3,NaN,2.19,1.0,0.0
4,NaN,2.02,0.0,1.0


In [12]:
data_model_hot_encoded.to_csv(f'{datapath}/02_intermediate/data_model_hot_encoded.csv', index=False)
data_predictive_hot_encoded.to_csv(f'{datapath}/02_intermediate/data_predictive_hot_encoded.csv', index=False)
data_prescriptive_hot_encoded.to_csv(f'{datapath}/02_intermediate/data_prescriptive_hot_encoded.csv', index=False)

In [13]:
data_model_ordinal_encoded = pd.DataFrame(data_model_ordinal_encoded, columns=['VAR_2'])
data_model.drop('VAR_2', axis=1, inplace=True)
data_model = pd.concat([data_model, data_model_ordinal_encoded], axis=1)
data_model.head()

,QTD_CHOC,VAR_1,PESO_BOMBOM,VAR_2
0,320.15,0.88,10.144919,0.0
1,320.14,1.55,11.547027,0.0
2,284.85,2.97,11.112109,0.0
3,263.92,2.36,9.900123,2.0
4,276.22,2.98,10.079822,2.0


In [14]:
data_predictive_ordinal_encoded = pd.DataFrame(data_predictive_ordinal_encoded, columns=['VAR_2'])
data_predictive.drop('VAR_2', axis=1, inplace=True)
data_predictive = pd.concat([data_predictive, data_predictive_ordinal_encoded], axis=1)
data_predictive.head()

,QTD_CHOC,VAR_1,PESO_BOMBOM,VAR_2
0,286.63,2.72,NaN,0.0
1,257.27,1.97,NaN,1.0
2,353.16,2.64,NaN,0.0
3,317.78,2.76,NaN,0.0
4,248.10,2.63,NaN,2.0


In [15]:
data_prescriptive_ordinal_encoded = pd.DataFrame(data_prescriptive_ordinal_encoded, columns=['VAR_2'])
data_prescriptive.drop('VAR_2', axis=1, inplace=True)
data_prescriptive = pd.concat([data_prescriptive, data_prescriptive_ordinal_encoded], axis=1)
data_prescriptive.head()

,QTD_CHOC,VAR_1,VAR_2
0,NaN,2.07,2.0
1,NaN,2.94,1.0
2,NaN,2.53,1.0
3,NaN,2.19,1.0
4,NaN,2.02,2.0


In [16]:
data_model.to_csv(f'{datapath}/02_intermediate/data_model_ordinal_encoded.csv', index=False)
data_predictive.to_csv(f'{datapath}/02_intermediate/data_predictive_ordinal_encoded.csv', index=False)
data_prescriptive.to_csv(f'{datapath}/02_intermediate/data_prescriptive_ordinal_encoded.csv', index=False)